In [1]:

import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import os
import gc
import matplotlib.pyplot as plt
from tqdm import * 
import featuretools as ft
from sklearn.experimental import enable_iterative_imputer
#----------------核心模型----------------
from sklearn.linear_model import LogisticRegression
#----------------交叉验证----------------
from sklearn.model_selection import StratifiedKFold, KFold
#----------------评估指标----------------
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss
#----------------忽略报警----------------
import warnings
warnings.filterwarnings('ignore')


e:\anaconda\Lib\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 10.0.1 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [2]:
# 读取训练数据和测试数据
train_data_ads = pd.read_csv('./train/train_data_ads.csv')
train_data_feeds = pd.read_csv('./train/train_feeds.csv')

test_data_ads = pd.read_csv('./test/test_data_ads.csv')
test_data_feeds = pd.read_csv('./test/test_feeds.csv')

train_data_ads.head(10)


,log_id,label,user_id,age,sex,residence,city,city_rank,series_dev,series_group,...,ad_click_list_v001,ad_click_list_v002,ad_click_list_v003,ad_close_list_v001,ad_close_list_v002,ad_close_list_v003,pt_d,u_newsCatInterestsST,u_refreshTimes,u_feedLifeCycle
0,576480,0,1516217,5,2,32,179,5,34,7,...,14737^25802^32492^13240^33873,1126^1037^1730^1719^1382,305^327^344^317^301,24107,1218,173,20191111,16^173^168^151^65,0,16
1,576478,0,1516217,5,2,32,179,5,34,7,...,14737^25802^32492^13240^33873,1126^1037^1730^1719^1382,305^327^344^317^301,24107,1218,173,20191111,16^173^168^151^65,0,16
2,576475,0,1516217,5,2,32,179,5,34,7,...,14737^25802^32492^13240^33873,1126^1037^1730^1719^1382,305^327^344^317^301,24107,1218,173,20191111,16^173^168^151^65,0,16
3,576473,0,1516217,5,2,32,179,5,34,7,...,14737^25802^32492^13240^33873,1126^1037^1730^1719^1382,305^327^344^317^301,24107,1218,173,20191111,16^173^168^151^65,0,16
4,576406,0,1516217,5,2,32,179,5,34,7,...,14737^25802^32492^13240^33873,1126^1037^1730^1719^1382,305^327^344^317^301,24107,1218,173,20191111,16^173^168^151^65,0,16
5,576488,0,1516217,5,2,32,179,5,34,7,...,14737^25802^32492^13240^33873,1126^1037^1730^1719^1382,305^327^344^317^301,24107,1218,173,20191111,16^173^168^151^65,0,16
6,576490,0,1516217,5,2,32,179,5,34,7,...,14737^25802^32492^13240^33873,1126^1037^1730^1719^1382,305^327^344^317^301,24107,1218,173,20191111,16^173^168^151^65,0,16
7,576342,0,1516217,5,2,32,179,5,34,7,...,14737^25802^32492^13240^33873,1126^1037^1730^1719^1382,305^327^344^317^301,24107,1218,173,20191111,16^173^168^151^65,0,16
8,576349,0,1516217,5,2,32,179,5,34,7,...,14737^25802^32492^13240^33873,1126^1037^1730^1719^1382,305^327^344^317^301,24107,1218,173,20191111,16^173^168^151^65,0,16
9,576351,0,1516217,5,2,32,179,5,34,7,...,14737^25802^32492^13240^33873,1126^1037^1730^1719^1382,305^327^344^317^301,24107,1218,173,20191111,16^173^168^151^65,0,16


In [3]:
train_data_feeds.head(10)

,u_userId,u_phonePrice,u_browserLifeCycle,u_browserMode,u_feedLifeCycle,u_refreshTimes,u_newsCatInterests,u_newsCatDislike,u_newsCatInterestsST,u_click_ca2_news,...,e_ch,e_m,e_po,e_pl,e_rn,e_section,e_et,label,cillabel,pro
0,1519036,14,17,14,17,7,78^0^112^57^94,0,78^98^98^168^112,78^98^168^112^8,...,19,24,10,3047,1,0,20191116,1,-1,40
1,1519036,14,17,14,17,7,78^0^112^57^94,0,78^98^112^8^94,78^98^112^8^94,...,19,24,2,3047,4,0,20191116,-1,-1,0
2,1519036,14,17,14,17,7,78^0^112^57^94,0,78^98^112^8^94,78^98^112^8^94,...,19,24,5,3047,2,0,20191116,-1,-1,0
3,1519036,14,17,14,17,7,78^0^112^57^94,0,78^112^8^94^98,112^98^78^8^94,...,19,24,6,275,1,1,20191116,-1,-1,0
4,1519036,14,17,14,17,7,78^0^112^94^57,0,78^98^98^98^17,17^98^78^168^112,...,19,24,6,3047,1,1,20191116,1,-1,60
5,1519036,14,17,14,17,7,78^0^112^57^94,0,78^98^112^8^94,78^98^112^8^94,...,19,24,7,3047,4,0,20191116,-1,-1,0
6,1519036,14,17,14,17,7,78^0^112^57^94,0,78^112^8^94^98,112^98^78^8^94,...,19,24,11,275,1,0,20191116,-1,-1,0
7,1519036,14,17,14,17,7,78^0^112^57^94,0,78^98^112^8^94,78^98^112^8^94,...,19,24,5,3047,4,0,20191116,-1,-1,0
8,1519036,14,17,14,17,7,78^0^112^57^94,0,78^112^8^94^98,112^98^78^8^94,...,19,24,6,275,1,1,20191116,1,-1,100
9,1519036,14,17,14,17,7,78^0^112^57^94,0,78^112^8^94^98,112^98^78^8^94,...,19,24,7,275,1,1,20191116,-1,-1,0


In [4]:
# 合并数据
train_data_ads['istest'] = 0
test_data_ads['istest'] = 1
data_ads = pd.concat([train_data_ads, test_data_ads], axis=0, ignore_index=True)

train_data_feeds['istest'] = 0
test_data_feeds['istest'] = 1
data_feeds = pd.concat([train_data_feeds, test_data_feeds], axis=0, ignore_index=True)

del train_data_ads, test_data_ads, train_data_feeds, test_data_feeds
gc.collect()

20

In [5]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [6]:
# 自然数编码
def label_encode(series, series2):
    unique = list(series.unique())
    return series2.map(dict(zip(
        unique, range(series.nunique())
    )))

for col in ['ad_click_list_v001','ad_click_list_v002','ad_click_list_v003','ad_close_list_v001','ad_close_list_v002','ad_close_list_v003','u_newsCatInterestsST']:
    data_ads[col] = label_encode(data_ads[col], data_ads[col])

In [7]:
%%time
gap_max = 6
gap_list = list(range(1, gap_max+1))  #穿越的间隔
print(f'间隔列表：{gap_list}')
print(data_ads.columns)

# 要找哪些不适合构建穿越特征
col_ads_deprecated = ['label', 'istest','age','sex','pt_d', 'log_id','residence','city','city_rank',
                      'series_dev','series_group','emui_dev','device_name','device_size', 'site_id',
                       'ad_close_list_v002', 'ad_close_list_v003', 'net_type',
                       'u_feedLifeCycle','hispace_app_tags', 'app_second_class', 'app_score',
                      ]
cols = [f for f in data_ads.columns if f not in col_ads_deprecated]
print(f'用来构建穿越特征的广告域变量有{len(cols)}个，分别是：{cols}')

print(f'data_ads.shape = {data_ads.shape}')
print(f'data_feeds.shape = {data_feeds.shape}')

## 构建穿越特征
for col in tqdm(cols):
    for gap in gap_list:
        tmp = data_ads.groupby([col])['pt_d'].shift(-gap)  #往左移动，即未来时刻的数据
        data_ads['ts_{}_{}_diff_last'.format(col, gap)] = tmp - data_ads['pt_d']   #上一条样本到本条样本的时间差

    for gap in gap_list:
        tmp = data_ads.groupby([col])['pt_d'].shift(+gap)  #往右移动，即过去时刻的数据
        data_ads['ts_{}_{}_diff_next'.format(col, gap)] = tmp - data_ads['pt_d']   #上一条样本到本条样本的时间差


间隔列表：[1, 2, 3, 4]
Index(['log_id', 'label', 'user_id', 'age', 'sex', 'residence', 'city',
       'city_rank', 'series_dev', 'series_group', 'device_dev', 'device_name',
       'device_size', 'net_type', 'task_id', 'adv_id', 'creat_type_cd',
       'adv_prim_id', 'inter_type_cd', 'slot_id', 'site_id', 'spread_app_id',
       'hispace_app_tags', 'app_second_class', 'app_score',
       'ad_click_list_v001', 'ad_click_list_v002', 'ad_click_list_v003',
       'ad_close_list_v001', 'ad_close_list_v002', 'ad_close_list_v003',
       'pt_d', 'u_newsCatInterestsST', 'u_refreshTimes', 'u_feedLifeCycle',
       'istest'],
      dtype='object')
用来构建穿越特征的广告域变量有15个，分别是：['user_id', 'device_dev', 'task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id', 'inter_type_cd', 'slot_id', 'spread_app_id', 'ad_click_list_v001', 'ad_click_list_v002', 'ad_click_list_v003', 'ad_close_list_v001', 'u_newsCatInterestsST', 'u_refreshTimes']
data_ads.shape = (2930502, 36)
data_feeds.shape = (970782, 29)


100%|██████████| 15/15 [00:05<00:00,  2.87it/s]

CPU times: total: 5.25 s
Wall time: 5.25 s


In [ ]:
# 压缩使用内存
data_ads = reduce_mem_usage(data_ads)
print(f'data_ads.shape = {data_ads.shape}')
# Mem. usage decreased to 2351.47 Mb (69.3% reduction)

In [ ]:
print(f'data_feeds.shape = ',data_feeds.shape)
print(f'data_ads.shape = ',data_ads.shape)
print('信息域特征：')
print(data_feeds.columns)

## 去重计数特征
cols = [f for f in data_feeds.columns if f not in [ 'u_phonePrice', 'u_browserLifeCycle', 'u_browserMode',
       'u_feedLifeCycle', 'u_refreshTimes', 'u_newsCatInterests','u_newsCatDislike','i_dislikeTimes','u_userId', 
       'i_dtype', 'e_ch', 'e_m', 'e_pl', 'e_rn', 'e_section', 'label', 'cillabel', 'pro', 'istest']]
print(f'用来构建nunique特征的源域变量有{len(cols)}个，分别是：{cols}')
for col in tqdm(cols):
    tmp = data_feeds.groupby(['u_userId'])[col].nunique().reset_index()  #无重复值
    tmp.columns = ['user_id', col+'_feeds_nuni']
    data_ads = data_ads.merge(tmp, on='user_id', how='left')

## 均值特征
cols = [f for f in data_feeds.columns if f in ['i_upTimes', 'u_refreshTimes']]
print(f'用来构建mean特征的源域变量有{len(cols)}个，分别是：{cols}')
for col in tqdm(cols):
    tmp = data_feeds.groupby(['u_userId'])[col].mean().reset_index()
    tmp.columns = ['user_id', col+'_feeds_mean']
    data_ads = data_ads.merge(tmp, on='user_id', how='left')

print(f'data_feeds.shape = ',data_feeds.shape)
print(f'data_ads.shape = ',data_ads.shape)

In [ ]:
# 压缩使用内存
print(f'data_ads.shape = {data_ads.shape}')
print(f'data_feeds.shape = {data_feeds.shape}')
data_ads = reduce_mem_usage(data_ads)
# Mem. usage decreased to 2351.47 Mb (69.3% reduction)

In [ ]:
# 划分训练集和测试集
cols = [f for f in data_ads.columns if f not in ['label','istest']]
X_train = data_ads[data_ads.istest==0][cols]
X_test = data_ads[data_ads.istest==1][cols]
Y_train = data_ads[data_ads.istest==0]['label']

print('X_train.shape = ', X_train.shape)
print('Y_train.shape = ', Y_train.shape)
print('X_test.shape = ', X_test.shape)

del data_ads, data_feeds
gc.collect()

val_counts = Y_train.value_counts()
print(val_counts)
ratio = val_counts[1] / len(Y_train)
print(f'正样本个数：{val_counts[1]}，样本总数：{Y_train.shape[0]}，点击率：{ratio*100}%')

In [ ]:
def cv_model(train_x, train_y, test_x, clf_name, seed=2022):
    # 处理数据中的缺失值
    imputer = SimpleImputer(strategy='mean')
    train_x_imputed = imputer.fit_transform(train_x)
    test_x_imputed = imputer.transform(test_x)

    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
    train = np.zeros(train_x.shape[0])
    test = np.zeros(test_x.shape[0])
    cv_scores = []

    for i, (train_index, valid_index) in enumerate(kf.split(train_x_imputed, train_y)):
        print('************************************ 第 {} 折 ************************************'.format(str(i+1)))
        trn_x, trn_y, val_x, val_y = train_x_imputed[train_index], train_y[train_index], train_x_imputed[valid_index], train_y[valid_index]

        model = LogisticRegression()
        model.fit(trn_x, trn_y)

        val_pred = model.predict_proba(val_x)[:, 1]
        test_pred = model.predict_proba(test_x_imputed)[:, 1]

        train[valid_index] = val_pred
        test += test_pred / kf.n_splits
        cv_scores.append(roc_auc_score(val_y, val_pred))

        print('第 ', i+1, ' 折，cv_scores: ', cv_scores)

    print("%s 评分列表:" % clf_name, cv_scores)
    print("%s 平均评分:" % clf_name, np.mean(cv_scores))
    print("%s 评分标准差:" % clf_name, np.std(cv_scores))
    return train, test


In [ ]:
%%time
# 调用 cv_model 函数，并传递Logistic Regression模型
logreg_train, logreg_test = cv_model(X_train, Y_train, X_test, "Logistic Regression")

In [ ]:
X_test['pctr'] = logreg_test
X_test[['log_id','pctr']].to_csv('submission.csv', index=False)
print('Done.')